In [3]:
from preprocessEMG import train_valid_test_split, getXY
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from statistics import mean, stdev
import numpy as np

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier


from exportCSV import exportCSV

In [4]:
training, validation, testing = train_valid_test_split()

In [8]:
def selectLimitFeatures(RMS):
    indices = [1, 3, 5, 7]
    limitedRMS = []
    for item in RMS:
        limitedRMS.append([item[index] for index in indices])
    return limitedRMS

def evaluate(clf, clf_name):
    f1_macro = []
    f1_micro = []
    accuracy = []
    combineTrainX = []
    combineTrainY = []
    for i in range(36):
        trainRMSX, trainX, trainY = getXY(training[i])
        validRMSX, validX, validY = getXY(validation[i])
        # combine validatation and training together
        trainRMSX.extend(validRMSX)
        trainX.extend(validX)
        trainY.extend(validY)
        # use limited features
        trainRMSX = selectLimitFeatures(trainRMSX)
        combineTrainX.extend(trainRMSX)
        combineTrainY.extend(trainY)
    # train the model
    clf.fit(combineTrainX, combineTrainY)
    for i in range(36):
        testRMSX, testX, testY = getXY(testing[i])
        testRMSX = selectLimitFeatures(testRMSX)
        # test the model
        predictY = clf.predict(testRMSX)
        f1_macro.append(f1_score(testY, predictY, average='macro'))
        accuracy.append(accuracy_score(testY, predictY))

    print("Macro-F1: {}, Accuracy: {}".format(mean(f1_macro), mean(accuracy)))
    f1_scores_2 = [clf_name, mean(f1_macro), stdev(f1_macro)]
    f1_scores_2.extend(f1_macro)
    accuracies_2 = [clf_name, mean(accuracy), stdev(accuracy)]
    accuracies_2.extend(accuracy)
    exportCSV(f1_scores_2, "limited_f1_macro_agg.csv")
    exportCSV(accuracies_2, "limited_accuracy_agg.csv")

In [9]:
clf = RandomForestClassifier(n_estimators=300, random_state=0, n_jobs=-1) # 1
evaluate(clf, "randomforest")

Macro-F1: 0.7879386516280203, Accuracy: 0.793594809607248


In [10]:
clf = ExtraTreesClassifier(n_estimators=200, random_state=0, n_jobs=-1) # 2
evaluate(clf, "extratrees") 

Macro-F1: 0.7831057277676164, Accuracy: 0.7865651217737695


In [14]:
clf = MLPClassifier(hidden_layer_sizes=(100), max_iter=200, random_state=9) # 3
evaluate(clf, "mlp")

/home/yuhang/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Macro-F1: 0.7772038556293608, Accuracy: 0.7826646630961962


In [12]:
clf = KNeighborsClassifier(5) # 4
evaluate(clf, "knn")

Macro-F1: 0.7563952336078644, Accuracy: 0.7596272076411694
